In [2]:
import zipfile
import os
import pandas as pd
from google.colab import files
from pathlib import Path

# Step 1: Upload image.zip and labels.zip
uploaded = files.upload()  # Expecting: images.zip and labels.zip

# Step 2: Define target directories
base_dir = Path("dataset")
images_dir = base_dir / "images"
labels_dir = base_dir / "labels"

# Step 3: Create directories
images_dir.mkdir(parents=True, exist_ok=True)
labels_dir.mkdir(parents=True, exist_ok=True)

# Step 4: Unzip and flatten
def extract_zip(zip_file, target_dir):
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(target_dir)

    # Move files from subfolders to the root of target_dir
    for root, _, files in os.walk(target_dir):
        for f in files:
            src = os.path.join(root, f)
            dst = os.path.join(target_dir, f)
            if src != dst:
                os.rename(src, dst)

    # Cleanup subfolders
    for item in os.listdir(target_dir):
        item_path = os.path.join(target_dir, item)
        if os.path.isdir(item_path):
            os.rmdir(item_path)

extract_zip("images.zip", images_dir)
extract_zip("labels.zip", labels_dir)

# Step 5: Process files
image_data = []

for img_file in os.listdir(images_dir):
    if not img_file.lower().endswith((".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff")):
        continue

    image_name = Path(img_file).stem
    label_file = labels_dir / f"{image_name}.txt"
    image_path = images_dir / img_file

    if not label_file.exists():
        print(f"Skipping '{img_file}' (no label found).")
        os.remove(image_path)
        continue

    # Read label file
    with open(label_file, "r") as file:
        lines = [l.strip() for l in file.readlines() if l.strip()]

    tumor_flag = 0
    box_list = []

    for line in lines:
        parts = line.split()
        if len(parts) != 5:
            continue
        class_id, *coords = parts
        if class_id == '1':
            try:
                box = [float(x) for x in coords]
                box_list.append(box)
                tumor_flag = 1
            except ValueError:
                print(f" Invalid format in {label_file.name}: {line}")
                continue

    image_data.append({
        "filename": img_file,
        "has_tumor": tumor_flag,
        "boxes": box_list
    })

# Step 6: Save DataFrame
df = pd.DataFrame(image_data, columns=["filename", "has_tumor", "boxes"])
df.to_csv("tumor_data.csv", index=False)
print("\n Saved processed metadata to 'tumor_data.csv'")
print(" - 1 = tumor present, 0 = no tumor")
print(" - Only bounding boxes of class 1 are retained.")


Saving images.zip to images.zip
Saving labels.zip to labels (1).zip
Skipping '72 (12).jpg' (no label found).
Skipping '00364_120.jpg' (no label found).
Skipping '00364_113.jpg' (no label found).
Skipping '00364_107.jpg' (no label found).
Skipping '00364_119.jpg' (no label found).
Skipping '00360_108.jpg' (no label found).
Skipping '00364_125.jpg' (no label found).
Skipping '00360_129.jpg' (no label found).
Skipping '00364_127.jpg' (no label found).
Skipping '00360_120.jpg' (no label found).
Skipping '00406_98.jpg' (no label found).
Skipping '00360_126.jpg' (no label found).
Skipping '00360_122.jpg' (no label found).
Skipping '00360_114.jpg' (no label found).
Skipping '00360_115.jpg' (no label found).

 Saved processed metadata to 'tumor_data.csv'
 - 1 = tumor present, 0 = no tumor
 - Only bounding boxes of class 1 are retained.


In [4]:
import os
import zipfile
import cv2
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt

from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# ========== Step 1: Unzip Image and Label Files ==========
import shutil

def unzip_and_flatten(zip_path, extract_to):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    for root, _, files in os.walk(extract_to):
        for file in files:
            src = os.path.join(root, file)
            dst = os.path.join(extract_to, file)
            if src != dst:
                shutil.move(src, dst)
    for subfolder in os.listdir(extract_to):
        subfolder_path = os.path.join(extract_to, subfolder)
        if os.path.isdir(subfolder_path):
            shutil.rmtree(subfolder_path)

unzip_and_flatten("images.zip", "images_data")
unzip_and_flatten("labels.zip", "labels_data")

# ========== Step 2: Process Data ==========
IMG_DIM = 512
image_dir = "images_data"
label_dir = "labels_data"

inputs, labels = [], []

for fname in os.listdir(label_dir):
    label_path = os.path.join(label_dir, fname)
    image_path = os.path.join(image_dir, fname.replace('.txt', '.jpg'))

    if not os.path.exists(image_path):
        continue

    with open(label_path, 'r') as file:
        lines = file.readlines()

    has_tumor = any(line.startswith("1") for line in lines if len(line.split()) == 5)
    labels.append(1 if has_tumor else 0)

    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        continue

    # Resize with padding
    h, w = img.shape
    scale = IMG_DIM / max(h, w)
    new_h, new_w = int(h * scale), int(w * scale)
    resized = cv2.resize(img, (new_w, new_h))
    pad_top = (IMG_DIM - new_h) // 2
    pad_bottom = IMG_DIM - new_h - pad_top
    pad_left = (IMG_DIM - new_w) // 2
    pad_right = IMG_DIM - new_w - pad_left
    padded_img = cv2.copyMakeBorder(resized, pad_top, pad_bottom, pad_left, pad_right,
                                    cv2.BORDER_CONSTANT, value=0)

    # Convert to 3-channel and preprocess
    padded_img = np.expand_dims(padded_img, axis=-1)
    padded_img = np.repeat(padded_img, 3, axis=-1)
    padded_img = preprocess_input(padded_img.astype(np.float32))
    inputs.append(padded_img)

X = np.array(inputs)
y = np.array(labels)

# ========== Step 3: Train/Val Split ==========
X_train, X_val, y_train, y_val = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# ========== Step 4: Data Augmentation ==========
augment = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomContrast(0.1),
])

# ========== Step 5: Class Weights ==========
weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
weights = dict(enumerate(weights))

# ========== Step 6: Build Model ==========
base = EfficientNetB0(include_top=False, input_shape=(IMG_DIM, IMG_DIM, 3), weights="imagenet")
base.trainable = True
for layer in base.layers[:-30]:  # freeze most layers
    layer.trainable = False

inp = layers.Input(shape=(IMG_DIM, IMG_DIM, 3))
x = augment(inp)
x = base(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.3)(x)
out = layers.Dense(1, activation='sigmoid')(x)

model = models.Model(inputs=inp, outputs=out)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
)

# ========== Step 7: Train ==========
callbacks = [
    EarlyStopping(patience=4, monitor='val_loss', restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1),
    ModelCheckpoint("optimized_classifier.keras", save_best_only=True, monitor='val_loss', verbose=1)
]

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=32,
    class_weight=weights,
    callbacks=callbacks
)

# ========== Step 8: Evaluation ==========
y_pred = (model.predict(X_val) > 0.5).astype(int)
print("\n Classification Report:")
print(classification_report(y_val, y_pred, digits=4))


Epoch 1/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step - accuracy: 0.5284 - loss: 0.6865 - precision_1: 0.5574 - recall_1: 0.4363
Epoch 1: val_loss improved from inf to 0.66233, saving model to optimized_classifier.keras
22/22 ━━━━━━━━━━━━━━━━━━━━ 23s 350ms/step - accuracy: 0.5303 - loss: 0.6860 - precision_1: 0.5587 - recall_1: 0.4441 - val_accuracy: 0.5909 - val_loss: 0.6623 - val_precision_1: 0.5746 - val_recall_1: 0.8370 - learning_rate: 1.0000e-04
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step - accuracy: 0.6658 - loss: 0.6247 - precision_1: 0.6894 - recall_1: 0.6972
Epoch 2: val_loss improved from 0.66233 to 0.64047, saving model to optimized_classifier.keras
22/22 ━━━━━━━━━━━━━━━━━━━━ 4s 173ms/step - accuracy: 0.6660 - loss: 0.6247 - precision_1: 0.6896 - recall_1: 0.6955 - val_accuracy: 0.6136 - val_loss: 0.6405 - val_precision_1: 0.6017 - val_recall_1: 0.7717 - learning_rate: 1.0000e-04
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step - accuracy: 0.6512 - loss: 0.6170 - p

In [5]:
#  Required Libraries
import os
import zipfile
import shutil
from pathlib import Path
from sklearn.model_selection import train_test_split

# Unzip Data
def unzip_and_flatten(zip_path, target_dir):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(target_dir)

    # Flatten nested dirs if any
    for root, _, files in os.walk(target_dir):
        for file in files:
            src = os.path.join(root, file)
            dst = os.path.join(target_dir, file)
            if src != dst:
                shutil.move(src, dst)
    # Cleanup folders
    for f in os.listdir(target_dir):
        p = os.path.join(target_dir, f)
        if os.path.isdir(p): shutil.rmtree(p)

# Input ZIP files
image_zip = "images.zip"
label_zip = "labels.zip"

# Extraction directories
raw_dir = Path("raw_data")
img_dir = raw_dir / "images"
lbl_dir = raw_dir / "labels"
shutil.rmtree(raw_dir, ignore_errors=True)
img_dir.mkdir(parents=True)
lbl_dir.mkdir(parents=True)

# Extract
unzip_and_flatten(image_zip, img_dir)
unzip_and_flatten(label_zip, lbl_dir)

# Split and Filter
image_list = sorted([f for f in img_dir.glob("*.jpg")])
train_files, val_files = train_test_split(image_list, test_size=0.2, random_state=42)

# YOLO-style structure
base = Path("yolo_dataset")
for folder in ['images/train', 'images/val', 'labels/train', 'labels/val']:
    (base / folder).mkdir(parents=True, exist_ok=True)

def prepare_split(images, split):
    for img_path in images:
        name = img_path.stem
        label_path = lbl_dir / f"{name}.txt"
        if not label_path.exists():
            continue

        with open(label_path, 'r') as f:
            lines = f.readlines()

        tumor_labels = []
        for line in lines:
            parts = line.strip().split()
            if parts and parts[0] == "1":
                parts[0] = "0"
                tumor_labels.append(" ".join(parts))

        if tumor_labels:
            shutil.copy(img_path, base / f"images/{split}" / img_path.name)
            with open(base / f"labels/{split}" / f"{name}.txt", "w") as f:
                f.write("\n".join(tumor_labels))

prepare_split(train_files, "train")
prepare_split(val_files, "val")

# Generate YAML
dataset_yaml = f"""
path: {base.resolve()}
train: images/train
val: images/val
names:
  0: tumor
"""
yaml_path = base / "tumor_data.yaml"
with open(yaml_path, "w") as f:
    f.write(dataset_yaml.strip())

# Train YOLOv8
!pip install -q ultralytics
from ultralytics import YOLO
import pandas as pd

model = YOLO("yolov8s.pt")  # we can also use yolov8m.pt for better performance

model.train(
    data=str(yaml_path),
    epochs=150,
    imgsz=512,
    batch=16,
    lr0=0.002,
    optimizer='AdamW',
    weight_decay=0.0005,
    dropout=0.2,
    patience=20,
    mixup=0.1,
    cos_lr=True,
    auto_augment='randaugment',
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    translate=0.1,
    scale=0.4,
    shear=2.0,
    perspective=0.0005,
    flipud=0.1,
    fliplr=0.5,
    mosaic=1.0,
    label_smoothing=0.01,
    warmup_epochs=5,
    box=0.05,
    cls=0.3,
    dfl=1.5,
    project='tumor_detection_runs',
    name='custom_detector',
    seed=42,
    device='cuda'
)

#  Evaluation
best_model = YOLO("tumor_detection_runs/custom_detector/weights/best.pt")
metrics = best_model.val()

# Save results
f1 = 2 * (metrics.box.mp * metrics.box.mr) / (metrics.box.mp + metrics.box.mr + 1e-6)
results = {
    "Precision": metrics.box.mp,
    "Recall": metrics.box.mr,
    "F1 Score": f1,
    "mAP@0.5": metrics.box.map50,
    "mAP@0.5:0.95": metrics.box.map
}
df = pd.DataFrame([results])
df.to_csv("bbox_eval_metrics.csv", index=False)

#  Display Metrics Summary
print("\n Evaluation Metrics:")
print(f" - Precision      : {metrics.box.mp:.4f}")
print(f" - Recall         : {metrics.box.mr:.4f}")
print(f" - F1 Score       : {f1:.4f}")
print(f" - mAP@0.5        : {metrics.box.map50:.4f}")
print(f" - mAP@0.5:0.95   : {metrics.box.map:.4f}")

# Treat F1 as pseudo-accuracy for interpretation
print(f"\n Pseudo-Accuracy (F1-Score as proxy): {f1:.4f}")

# Predict
best_model.predict(source=base / "images/val", save=True, conf=0.25)
print(" Inference results saved to: runs/detect/predict")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 994.0/994.0 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 115.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 97.0 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ul

100%|██████████| 21.5M/21.5M [00:00<00:00, 178MB/s]


WARNING ⚠️ 'label_smoothing' is deprecated and will be removed in in the future.
Ultralytics 8.3.105 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=yolo_dataset/tumor_data.yaml, epochs=150, time=None, patience=20, batch=16, imgsz=512, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=tumor_detection_runs, name=custom_detector, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.2, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_fra

100%|██████████| 755k/755k [00:00<00:00, 14.4MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics

...


55 epochs completed in 0.044 hours.
Optimizer stripped from tumor_detection_runs/custom_detector/weights/last.pt, 22.5MB
Optimizer stripped from tumor_detection_runs/custom_detector/weights/best.pt, 22.5MB

Validating tumor_detection_runs/custom_detector/weights/best.pt...
Ultralytics 8.3.105 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:00<00:00,  5.00it/s]


                   all         91         93      0.941      0.892      0.946      0.697
Speed: 0.1ms preprocess, 0.6ms inference, 0.0ms loss, 2.1ms postprocess per image
Results saved to tumor_detection_runs/custom_detector
Ultralytics 8.3.105 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs


val: Scanning /content/yolo_dataset/labels/val.cache... 91 images, 0 backgrounds, 0 corrupt: 100%|██████████| 91/91 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:01<00:00,  5.44it/s]


                   all         91         93      0.941      0.892      0.945      0.695
Speed: 1.5ms preprocess, 2.9ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to runs/detect/val

 Evaluation Metrics:
 - Precision      : 0.9407
 - Recall         : 0.8925
 - F1 Score       : 0.9159
 - mAP@0.5        : 0.9449
 - mAP@0.5:0.95   : 0.6953

 Pseudo-Accuracy (F1-Score as proxy): 0.9159

image 1/91 /content/yolo_dataset/images/val/00056_239.jpg: 512x512 1 tumor, 8.0ms
image 2/91 /content/yolo_dataset/images/val/00058_179.jpg: 512x512 (no detections), 7.8ms
image 3/91 /content/yolo_dataset/images/val/00063_197.jpg: 512x512 1 tumor, 7.5ms
image 4/91 /content/yolo_dataset/images/val/00066_240.jpg: 512x512 1 tumor, 7.6ms
image 5/91 /content/yolo_dataset/images/val/00066_259.jpg: 512x512 1 tumor, 7.4ms
image 6/91 /content/yolo_dataset/images/val/00071_183.jpg: 512x512 1 tumor, 7.6ms
image 7/91 /content/yolo_dataset/images/val/00074_229.jpg: 512x512 1 tumor, 7.4ms
image 8/91

In [6]:
import os
import cv2
import numpy as np
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.efficientnet import preprocess_input
from ultralytics import YOLO
from tqdm import tqdm

# Configs
IMG_SIZE = 512
THRESHOLD = 0.5
TEST_DIR = "PATH"
CLASSIFIER_PATH = "/content/optimized_classifier.keras"
DETECTOR_PATH = "/content/yolov8s.pt"

#  Load models
clf_model = load_model(CLASSIFIER_PATH)
yolo_model = YOLO(DETECTOR_PATH)

#  Image preprocessing
def preprocess_image(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        return None
    h, w = img.shape
    scale = IMG_SIZE / max(h, w)
    new_w, new_h = int(w * scale), int(h * scale)
    resized = cv2.resize(img, (new_w, new_h))
    top = (IMG_SIZE - new_h) // 2
    bottom = IMG_SIZE - new_h - top
    left = (IMG_SIZE - new_w) // 2
    right = IMG_SIZE - new_w - left
    padded = cv2.copyMakeBorder(resized, top, bottom, left, right, cv2.BORDER_CONSTANT, value=0)
    rgb = np.repeat(padded[..., np.newaxis], 3, axis=-1)
    processed = preprocess_input(rgb.astype('float32'))
    return np.expand_dims(processed, axis=0)

#  Run Pipeline

image_paths = sorted([os.path.join(TEST_DIR, f) for f in os.listdir(TEST_DIR) if f.lower().endswith(('.jpg', '.png'))])
y_true, y_pred = [], []

print(f"\n Processing {len(image_paths)} test images...\n")

for path in tqdm(image_paths):
    img_name = os.path.basename(path)
    img_tensor = preprocess_image(path)

    if img_tensor is None:
        print(f" Skipped: {img_name} (could not load)")
        continue

    # Run classification
    prob = clf_model.predict(img_tensor, verbose=0)[0][0]
    label = int(prob >= THRESHOLD)
    y_pred.append(label)

    # Inferred true label from filename
    gt = 1 if "tumor" in img_name.lower() else 0
    y_true.append(gt)

    print(f" {img_name} | Tumor Probability: {prob:.4f} → {'Tumor' if label else 'No Tumor'}")

    # YOLO detection if tumor predicted
    if label:
        yolo_model.predict(source=path, save=True, conf=0.25)
        print(" Detection saved to: runs/detect/predict")

#  Results
print("\n Classification Report:")
print(classification_report(y_true, y_pred, digits=4))

acc = accuracy_score(y_true, y_pred)
print(f"\n Overall Classification Accuracy: {acc:.4f}")


'\nimage_paths = sorted([os.path.join(TEST_DIR, f) for f in os.listdir(TEST_DIR) if f.lower().endswith((\'.jpg\', \'.png\'))])\ny_true, y_pred = [], []\n\nprint(f"\n Processing {len(image_paths)} test images...\n")\n\nfor path in tqdm(image_paths):\n    img_name = os.path.basename(path)\n    img_tensor = preprocess_image(path)\n\n    if img_tensor is None:\n        print(f" Skipped: {img_name} (could not load)")\n        continue\n\n    # Run classification\n    prob = clf_model.predict(img_tensor, verbose=0)[0][0]\n    label = int(prob >= THRESHOLD)\n    y_pred.append(label)\n\n    # Inferred true label from filename\n    gt = 1 if "tumor" in img_name.lower() else 0\n    y_true.append(gt)\n\n    print(f" {img_name} | Tumor Probability: {prob:.4f} → {\'Tumor\' if label else \'No Tumor\'}")\n\n    # YOLO detection if tumor predicted\n    if label:\n        yolo_model.predict(source=path, save=True, conf=0.25)\n        print(" Detection saved to: runs/detect/predict")\n\n#  Results \npri